# PixelBytes: Catching Insights in Unified Multimodal Sequences

Ce notebook présente **PixelBytes**, un modèle innovant conçu pour générer simultanément du texte et des images pixel par pixel sous forme de séquences. L'objectif est d'explorer un embedding unifié qui permet une génération multimodale cohérente.

## Contexte et Architecture Proposée

### Fondements Théoriques
- **Image Transformer** : [Génération d'images pixel par pixel](https://arxiv.org/abs/1802.05751)
- **Bi-Mamba+** : [Modèle bidirectionnel pour la prévision de séries temporelles](https://arxiv.org/abs/2404.15772)
- **MambaByte** : [Modèle d'état d'espace sélectif sans token](https://arxiv.org/abs/2401.13660)

### Concept Clé
Le modèle PixelByte génère des séquences mixtes de texte et d'images. Il doit :
- Gérer les transitions entre texte et image avec des sauts de ligne (ASCII 0A).
- Maintenir la cohérence des dimensions des images générées.
- Assimiler la tâche de "copie" pour reproduire des motifs complexes.

Ce notebook utilise la puissance des GPU T4 x2 de Kaggle pour expérimenter avec des architectures avancées et des jeux de données volumineux, afin de relever les défis de la génération multimodale unifiée.

## Ressources du Projet

### Dataset
Pour ce projet, nous utiliserons le dataset **PixelBytes-Pokemon**, spécialement conçu pour cette tâche de génération multimodale. Ce dataset, créé par l'auteur de ce notebook, est disponible sur Hugging Face : [PixelBytes-Pokemon](https://huggingface.co/datasets/ffurfaro/PixelBytes-Pokemon). Il contient des séquences de texte et d'images de Pokémon, encodées de manière à permettre l'entraînement de notre modèle PixelByte sur des données multimodales.

### Implémentation
L'implémentation du modèle et les scripts d'entraînement sont disponibles dans le dépôt GitHub **Mamba-Bys** : [Mamba-Bys](https://github.com/fabienfrfr/Mamba-Bys). Ce dépôt contient le code source nécessaire pour reproduire les expériences, ainsi que des instructions détaillées sur la configuration et l'utilisation du modèle PixelByte.

## Tests à réaliser :

Nous utilisons les derniers modele avec la meilleure "train_loss" pour la génération de 16 sequences à la suite. Nous n'utiliserons pas le "best" accuracy test model, car celui-ci n'atteint jamais plus de 60% de reussite, contrairement au modele "last" qui atteint 90% de reussite en moyenne sur le train loss. La justification est que le modele, meme s'il a été entrainé sur les sequence qu'on utilise pour les mesure n'a jamais ete entrainé à généré 16 sequence à la suite, et encore moins pour les sequence en questions.

- hamming : Échelle : Généralement entre 0 et 1 pour des valeurs normalisées. Interprétation : Une valeur faible indique une grande similarité.
- cosine : Échelle : Entre -1 et 1, où 1 indique une similarité parfaite. Interprétation : Plus la valeur est proche de 1, plus les séquences sont similaires.
- bleu : Entre 0 et 1, où 1 est le meilleur score possible. Interprétation : Mesure la qualité de la traduction automatique ou la similarité de texte.


Observation : Les modèles transformers entrainé sont moyen car il n'y a pas le "trick" de l'encodage positionnel. De même, les evaluations de Mamba ont des resultats inferieurs que prevu par la loss et la précision du prochain mot par rapport à la prédiction à la suite de 16 valeurs (problème de mémoire ? surentrainement ?).


In [2]:
!pip install -q mamba-ssm causal-conv1d
!pip install -q git+https://github.com/fabienfrfr/PixelBytes.git@main

In [16]:
# no warning msg during eval
import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
from pixelbytes import *
from datasets import load_dataset
import numpy as np

In [4]:
hf_dataset = load_dataset("ffurfaro/PixelBytes-Pokemon")
hf_dataset

Generating train split:   0%|          | 0/964 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'caption', 'pixelbyte'],
        num_rows: 964
    })
})

In [5]:
for i in range(0,9,3) :
    display(hf_dataset['train']['image'][i])

In [6]:
config = EvaluateMetricConfig()#output_dir="models/eval_results")
data_columns = hf_dataset["train"]['pixelbyte']
tokenizer = PixelBytesTokenizer()
evaluator = EvaluateMetric(config, data_columns, tokenizer)

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [10]:
## Eval loop for LSTM
model_list = ["rnn_bi_center_81_dim_64_state_2_layer_last",
            "rnn_bi_pxby_81_dim_64_state_2_layer_last",
            "rnn_bi_pxby_conv_162_dim_64_state_2_layer_last",
            "rnn_bi_pxby_conv_36_dim_64_state_2_layer_last",
            "rnn_bi_pxby_conv_81_dim_128_state_2_layer_last",
            "rnn_bi_pxby_conv_81_dim_32_state_2_layer_last",
            "rnn_bi_pxby_conv_81_dim_64_state_1_layer_last",
            "rnn_bi_pxby_conv_81_dim_64_state_3_layer_last",
            "rnn_bi_pxby_noconv_81_dim_64_state_2_layer_last",
            "rnn_uni_pxby_conv_81_dim_64_state_2_layer_last",]

for model_name in model_list : 
    model = SimpleRNNModel.from_pretrained("ffurfaro/PixelBytes-Pokemon", subfolder=model_name)  # Your actual model here
    evaluator.reset(model)
    results = evaluator.evaluate(model_name)
    print(results.describe())

Evaluating rnn_bi_center_81_dim_64_state_2_layer_last: 100%|██████████| 964/964 [03:06<00:00,  5.18it/s]


                row         n_seq       hamming        cosine          bleu
count  13740.000000  13740.000000  13740.000000  13740.000000  13740.000000
mean     488.100728      6.923290      0.184993      0.888156      0.750153
std      278.791416      4.637646      0.074060      0.082874      0.093438
min        0.000000      0.000000      0.000000      0.466742      0.257558
25%      245.750000      3.000000      0.152778      0.829991      0.704604
50%      493.000000      7.000000      0.187500      0.898113      0.752018
75%      732.000000     10.000000      0.201389      0.960754      0.799490
max      963.000000     27.000000      0.548611      1.000000      1.000000


Evaluating rnn_bi_pxby_81_dim_64_state_2_layer_last: 100%|██████████| 964/964 [04:28<00:00,  3.59it/s]


                row         n_seq       hamming        cosine          bleu
count  13740.000000  13740.000000  13740.000000  13740.000000  13740.000000
mean     488.100728      6.923290      0.153174      0.901877      0.777343
std      278.791416      4.637646      0.061023      0.089961      0.083348
min        0.000000      0.000000      0.000000      0.475732      0.259487
25%      245.750000      3.000000      0.118056      0.832069      0.729718
50%      493.000000      7.000000      0.166667      0.920935      0.771137
75%      732.000000     10.000000      0.187500      0.983810      0.822703
max      963.000000     27.000000      0.541667      1.000000      1.000000


(…)62_dim_64_state_2_layer_last/config.json:   0%|          | 0.00/218 [00:00<?, ?B/s]

...

                row         n_seq       hamming        cosine          bleu
count  13740.000000  13740.000000  13740.000000  13740.000000  13740.000000
mean     488.100728      6.923290      0.150926      0.902716      0.779014
std      278.791416      4.637646      0.061842      0.090096      0.084106
min        0.000000      0.000000      0.000000      0.499448      0.258528
25%      245.750000      3.000000      0.111111      0.832700      0.730881
50%      493.000000      7.000000      0.159722      0.921280      0.771137
75%      732.000000     10.000000      0.187500      0.984747      0.824359
max      963.000000     27.000000      0.541667      1.000000      1.000000


(…)81_dim_64_state_2_layer_last/config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/349k [00:00<?, ?B/s]

Evaluating rnn_uni_pxby_conv_81_dim_64_state_2_layer_last: 100%|██████████| 964/964 [04:24<00:00,  3.64it/s]

                row         n_seq       hamming        cosine          bleu
count  13740.000000  13740.000000  13740.000000  13740.000000  13740.000000
mean     488.100728      6.923290      0.153372      0.904343      0.776722
std      278.791416      4.637646      0.063640      0.088132      0.087170
min        0.000000      0.000000      0.000000      0.503901      0.268475
25%      245.750000      3.000000      0.118056      0.836471      0.727813
50%      493.000000      7.000000      0.166667      0.922031      0.768687
75%      732.000000     10.000000      0.187500      0.984430      0.824359
max      963.000000     27.000000      0.555556      1.000000      1.000000


In [12]:
## Eval loop for Mamba
model_list = ["ssm_bi_pxby_conv_81_dim_64_state_1_layer_last",
            "ssm_bi_pxby_conv_81_dim_64_state_2_layer_last",
            "ssm_uni_pxby_conv_81_dim_64_state_2_layer_last",]

for model_name in model_list : 
    model = bMamba.from_pretrained("ffurfaro/PixelBytes-Pokemon", subfolder=model_name)  # Your actual model here
    evaluator.reset(model)
    results = evaluator.evaluate(model_name)
    print(results.describe())

(…)81_dim_64_state_1_layer_last/config.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/367k [00:00<?, ?B/s]

Evaluating ssm_bi_pxby_conv_81_dim_64_state_1_layer_last: 100%|██████████| 964/964 [07:13<00:00,  2.22it/s]


                row         n_seq       hamming        cosine          bleu
count  13740.000000  13740.000000  13740.000000  13740.000000  13740.000000
mean     488.100728      6.923290      0.170041      0.882871      0.753447
std      278.791416      4.637646      0.085658      0.105215      0.115439
min        0.000000      0.000000      0.000000      0.393256      0.248526
25%      245.750000      3.000000      0.125000      0.815702      0.716656
50%      493.000000      7.000000      0.173611      0.890871      0.758005
75%      732.000000     10.000000      0.201389      0.980778      0.811749
max      963.000000     27.000000      0.555556      1.000000      1.000000


(…)81_dim_64_state_2_layer_last/config.json:   0%|          | 0.00/209 [00:00<?, ?B/s]

...

                row         n_seq       hamming        cosine          bleu
count  13740.000000  13740.000000  13740.000000  13740.000000  13740.000000
mean     488.100728      6.923290      0.157960      0.896278      0.771450
std      278.791416      4.637646      0.073747      0.095151      0.096800
min        0.000000      0.000000      0.000000      0.443411      0.248526
25%      245.750000      3.000000      0.118056      0.828913      0.725988
50%      493.000000      7.000000      0.166667      0.910562      0.768687
75%      732.000000     10.000000      0.194444      0.982863      0.824359
max      963.000000     27.000000      0.555556      1.000000      1.000000


(…)81_dim_64_state_2_layer_last/config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/662k [00:00<?, ?B/s]

Evaluating ssm_uni_pxby_conv_81_dim_64_state_2_layer_last: 100%|██████████| 964/964 [06:51<00:00,  2.34it/s]

                row         n_seq       hamming        cosine          bleu
count  13740.000000  13740.000000  13740.000000  13740.000000  13740.000000
mean     488.100728      6.923290      0.166037      0.885845      0.760245
std      278.791416      4.637646      0.080592      0.101831      0.105805
min        0.000000      0.000000      0.000000      0.405492      0.248526
25%      245.750000      3.000000      0.125000      0.819435      0.718862
50%      493.000000      7.000000      0.173611      0.895194      0.761606
75%      732.000000     10.000000      0.201389      0.979766      0.815639
max      963.000000     27.000000      0.555556      1.000000      1.000000


In [14]:
## Eval loop for Mamba
model_list = ["attention_bi_pxby_conv_81_dim_64_state_1_layer_last",
            "attention_bi_pxby_conv_81_dim_64_state_2_layer_last",]

for model_name in model_list : 
    model = SimpleTransformerModel.from_pretrained("ffurfaro/PixelBytes-Pokemon", subfolder=model_name)  # Your actual model here
    evaluator.reset(model)
    results = evaluator.evaluate(model_name)
    print(results.describe())

(…)81_dim_64_state_1_layer_last/config.json:   0%|          | 0.00/225 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/384k [00:00<?, ?B/s]

Evaluating attention_bi_pxby_conv_81_dim_64_state_1_layer_last: 100%|██████████| 964/964 [04:13<00:00,  3.81it/s]


                row         n_seq       hamming        cosine          bleu
count  13740.000000  13740.000000  13740.000000  13740.000000  13740.000000
mean     488.100728      6.923290      0.157431      0.887452      0.764872
std      278.791416      4.637646      0.063624      0.102579      0.088029
min        0.000000      0.000000      0.000000      0.348762      0.248526
25%      245.750000      3.000000      0.118056      0.805195      0.711734
50%      493.000000      7.000000      0.173611      0.895911      0.756954
75%      732.000000     10.000000      0.201389      0.984724      0.813377
max      963.000000     27.000000      0.555556      1.000000      1.000000


(…)81_dim_64_state_2_layer_last/config.json:   0%|          | 0.00/225 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/697k [00:00<?, ?B/s]

Evaluating attention_bi_pxby_conv_81_dim_64_state_2_layer_last: 100%|██████████| 964/964 [05:35<00:00,  2.88it/s]

                row         n_seq       hamming        cosine          bleu
count  13740.000000  13740.000000  13740.000000  13740.000000  13740.000000
mean     488.100728      6.923290      0.158942      0.886944      0.759542
std      278.791416      4.637646      0.065899      0.103263      0.091884
min        0.000000      0.000000      0.000000      0.464571      0.248526
25%      245.750000      3.000000      0.118056      0.805237      0.704604
50%      493.000000      7.000000      0.173611      0.897269      0.749861
75%      732.000000     10.000000      0.201389      0.984671      0.810259
max      963.000000     27.000000      0.555556      1.000000      1.000000


In [15]:
ls eval_results

attention_bi_pxby_conv_81_dim_64_state_1_layer_last.csv
attention_bi_pxby_conv_81_dim_64_state_2_layer_last.csv
rnn_bi_center_81_dim_64_state_2_layer_last.csv
rnn_bi_pxby_81_dim_64_state_2_layer_last.csv
rnn_bi_pxby_conv_162_dim_64_state_2_layer_last.csv
rnn_bi_pxby_conv_36_dim_64_state_2_layer_last.csv
rnn_bi_pxby_conv_81_dim_128_state_2_layer_last.csv
rnn_bi_pxby_conv_81_dim_32_state_2_layer_last.csv
rnn_bi_pxby_conv_81_dim_64_state_1_layer_last.csv
rnn_bi_pxby_conv_81_dim_64_state_3_layer_last.csv
rnn_bi_pxby_noconv_81_dim_64_state_2_layer_last.csv
rnn_uni_pxby_conv_81_dim_64_state_2_layer_last.csv
ssm_bi_pxby_conv_81_dim_64_state_1_layer_last.csv
ssm_bi_pxby_conv_81_dim_64_state_2_layer_last.csv
ssm_uni_pxby_conv_81_dim_64_state_2_layer_last.csv
